In [1]:
%load_ext autoreload 
%aimport rugpeek_functions 
import matplotlib.pyplot as plt
import numpy as np
import lmfit as lf
import matplotlib as mpl
import sys

In [2]:
data_file_ferric_soret_shg0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_1_matrix"
data_file_ferric_soret_shg1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_2_matrix"
data_file_ferric_soret_shg2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_3_matrix"

data_file_ferric_soret_fun0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_1_matrix"
data_file_ferric_soret_fun1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_2_matrix"
data_file_ferric_soret_fun2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_3_matrix"

solvent_ferric_soret_shg = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_SHG_chirp_1_matrix"
solvent_ferric_soret_fun = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_FUN_chirp_1_matrix"

In [3]:
%autoreload 1
rp = rugpeek_functions
#JDP create an instance of the Rug class
ferric_soret_shg = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_shg0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_shg1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_shg2, ".dat")])

ferric_soret_fun = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_fun0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_fun1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_fun2, ".dat")])

solvent_soret_shg = rp.Rug(solvent_ferric_soret_shg, ".dat")
solvent_soret_fun = rp.Rug(solvent_ferric_soret_fun, ".dat")


#combined = rp.RugTools.combine_rugs_wavelengths([ferric_soret_shg, ferric_soret_fun])
#rp.RugTools.average_rugs([ferric_soret_shg, ferric_soret_fun])

Missing: Concentration
Missing: Concentration


In [4]:
%matplotlib qt5

#ferric_soret_shg = rp.Rug(data_file_ferric_soret_shg, ".dat")
#ferric_soret_fun = rp.Rug(data_file_ferric_soret_fun, ".dat")
#solvent_soret_shg = rp.Rug(solvent_ferric_soret_shg, ".dat")
#solvent_soret_fun = rp.Rug(solvent_ferric_soret_fun, ".dat")

solvent_soret_shg.get_dispersion_correction_eye(degree=2)
coefs_soret_shg = solvent_soret_shg.dispersion_coefs
solvent_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.offset(coef=coefs_soret_shg[-1])
#ferric_soret_shg.peek()

solvent_soret_fun.get_dispersion_correction_eye(degree=2)
coefs_soret_fun = solvent_soret_fun.dispersion_coefs
ferric_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)
solvent_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)

ferric_soret_fun.offset(coef=coefs_soret_fun[-1])
#ferric_soret_fun.peek()
plt.close('all')

Offset corrected
Offset corrected


In [5]:
import copy
ferric_soret_shg_mod = copy.deepcopy(ferric_soret_shg)
ferric_soret_fun_mod = copy.deepcopy(ferric_soret_fun)
solvent_soret_shg_mod = copy.deepcopy(solvent_soret_shg)
solvent_soret_fun_mod = copy.deepcopy(solvent_soret_fun)

ferric_soret_rugs = [ferric_soret_shg_mod, ferric_soret_fun_mod]
ferric_soret_total = rp.RugTools.combine_rugs_wavelengths(ferric_soret_rugs, "ferric_soret_combined_matrix")

solvent_soret_rugs = [solvent_soret_shg_mod, solvent_soret_fun_mod]
solvent_soret_total = rp.RugTools.combine_rugs_wavelengths(solvent_soret_rugs, "solvent_soret_combined_matrix")


In [6]:
ferric_soret_total.cut_wavelengths(401,414, fill=0)
ferric_soret_total.cut_wavelengths(512,518, fill=0)
ferric_soret_total.background_subtract()
ferric_soret_total.peek()


Taken average background
(239, 453) (239, 453)


In [100]:
solvent_soret_total.cut_wavelengths(400,418, fill=0)
solvent_soret_total.cut_wavelengths(512,518, fill=0)
#solvent_soret_total.background_subtract()
solvent_soret_total.peek()

In [ ]:
%autoreload 1
ferric_soret_total.background_subtract()

In [96]:
%autoreload 1
plt.close('all')
#ferric_soret_total.reset_matrix()


#ferric_soret_total.limit_times(tmin=10, tmax=1000)
#ferric_soret_total.limit_wavelengths(wlmin=413, wlmax=700)
                                                       
ferric_soret_total.compute_SVD(threshold=100)


#cpt1 = np.outer(ferric_soret_total.relevant_kinetics[2],
     #           ferric_soret_total.relevant_spectra[2])
'''
fig = plt.figure()
ax = fig.gca()
cmap='PuOr'
vmin = cpt1.min()
vmax = cpt1.max()
im = ax.pcolormesh(ferric_soret_total.wavelengths, ferric_soret_total.times, cpt1,
            cmap=cmap, norm=mpl.colors.TwoSlopeNorm(vcenter=0, vmin=vmin, vmax=vmax))
ax.set_yscale('symlog')
'''

plt.figure()
for i, spectrum in enumerate(ferric_soret_total.relevant_kinetics):
    plt.plot(ferric_soret_total.times, spectrum, label='Component '+str(i+1))

plt.xscale('symlog')
plt.legend()

plt.figure()
for i, spectrum in enumerate(ferric_soret_total.relevant_spectra):
    plt.plot(ferric_soret_total.wavelengths, spectrum, label='Component '+str(i+1))

plt.legend()


Taking threshold for singular value significance as 100x larger than smallest singular value (38.203806952).
Found 2 significant singular values
81.61 % of data variance described by component 1.
16.34 % of data variance described by component 2.
Overall, 97.95 % of data variance described by the first 2 components.
Leaving 2.05 % of data variance described by the the remaining 237 components which are below threshold (roughly 0.01 % per component).


In [30]:
plt.figure()
times = [400, 420, 430, 440, 450, 500, 600]
for time in times:
    plt.plot(ferric_soret_total.times, ferric_soret_total.get_time_trace(time), label=time)

plt.legend()

In [97]:
#### Fitting the actual spectra

%autoreload 2
t = ferric_soret_total.times

#plt.close('all')
def normalise(x):
    a = (x-np.min(x))/np.max(x)
    return a

def normalise_scale(x):
    return x/np.max(x)

def inverse_uncertainty(x, s):
    s_inv = s / (x**2)
    return s_inv
    
traces = ferric_soret_total.relevant_kinetics[0:3]

ncpts = 2

IRF_center_val = 0
IRF_center_min = -0.3
IRF_center_max = 0.3
IRF_center= [IRF_center_val, IRF_center_min, IRF_center_max]

IRF_sigma_val = 0.4
IRF_sigma_min = 0.05
IRF_sigma_max = 1
IRF_sigma= [IRF_sigma_val, IRF_sigma_min, IRF_sigma_max]

decay_amplitude_center = [1, 1, 1]
decay_amplitude_min = [-5, -5, -5]
decay_amplitude_max = [5, 5, 5]

decay_gamma_center = [1, 1, 1]
decay_gamma_min = [1E-6, 1E-6, 1E-6]
decay_gamma_max = [20, 20, 20]
mods = []

for idx, trace in enumerate(traces):
    y = normalise_scale(trace)
    print(f'Fitting component {idx+1} of the SVD.')
    print(f'Fitting to {ncpts} exponential decays.')
    
    for n in range(ncpts):
        if n == 0:
            mod = lf.models.ExponentialGaussianModel(prefix='p'+str(n)+'_')
            pars = mod.make_params(center = dict(value=IRF_center[0], min=IRF_center[1], 
                                                 max=IRF_center[2], vary=True),
                        sigma = dict(value=IRF_sigma[0], min=IRF_sigma[1], 
                                     max=IRF_sigma[2], vary=True),
                        amplitude = dict(value=decay_amplitude_center[n],min=decay_amplitude_min[n],
                                         max=decay_amplitude_max[n], vary=True), 
                        gamma = dict(value=decay_gamma_center[n], min=decay_gamma_min[n], 
                                     max=decay_gamma_max[n], vary=True))
            model = mod
        else:
            mod = lf.models.ExponentialGaussianModel(prefix='p'+str(n)+'_')
            pars.update(mod.make_params(center = dict(expr='p0_center', vary=False),
                        sigma = dict(expr='p0_center', vary=False),
                        amplitude = dict(value=decay_amplitude_center[n],min=decay_amplitude_min[n],
                                         max=decay_amplitude_max[n], vary=True), 
                        gamma = dict(value=decay_gamma_center[n], min=decay_gamma_min[n], 
                                     max=decay_gamma_max[n], vary=True)))
            model = model + mod


    out = model.fit(y, pars, x=t, method='least_squares')
    
    plt.figure()
    ax = out.plot_fit()
    ax.set_xlabel('Time (ps)')
    ax.set_ylabel(r'$\Delta OD')
    ax.set_xscale('symlog')
    print(f'Fitted IRF as centered at {t0} ps and with {FWHM} ps width.')
    for n in range(ncpts):
        l1 = np.round(1/out.params['p'+str(n)+'_gamma'].value, 4) 
        l1_err = np.round(inverse_uncertainty(l1, out.params['p'+str(n)+'_gamma'].stderr), 4)
        a1 = np.round(out.params['p'+str(n)+'_amplitude'].value,4)
        print(f'Lifetime is: {l1} +/- {l1_err} ps. Corresponding amplitude is {a1}.')


Fitting component 1 of the SVD.
Fitting to 2 exponential decays.
Fitted IRF as centered at 0.0826 ps and with 0.0567 ps width.
Lifetime is: 4.0805 +/- 0.0003 ps. Corresponding amplitude is 2.1928.
Lifetime is: 0.5347 +/- 0.2104 ps. Corresponding amplitude is 0.3704.
Fitting component 2 of the SVD.
Fitting to 2 exponential decays.
Fitted IRF as centered at 0.0826 ps and with 0.0567 ps width.
Lifetime is: 5.1488 +/- 0.0002 ps. Corresponding amplitude is -3.0872.
Lifetime is: 0.4866 +/- 0.1448 ps. Corresponding amplitude is 0.9824.


In [98]:
DAS1 = 2.1752*ferric_soret_total.singular_values[0]*ferric_soret_total.relevant_spectra[0] \
        -3.108*ferric_soret_total.singular_values[1]*ferric_soret_total.relevant_spectra[1]

DAS2 = 0.4311*ferric_soret_total.singular_values[0]*ferric_soret_total.relevant_spectra[0] \
     + 1.0255*ferric_soret_total.singular_values[1]*ferric_soret_total.relevant_spectra[1]

plt.figure()
plt.plot(ferric_soret_total.wavelengths, DAS1, label='DAS, 4.5ps cpt')
plt.plot(ferric_soret_total.wavelengths, DAS2, label='DAS, 0.5ps cpt')
plt.legend()

In [ ]:
ferric_soret_total_cut = copy.deepcopy(ferric_soret_total)

#ferric_soret_total_cut.cut_wavelengths(400,415)
ferric_soret_total_cut.cut_wavelengths(510, 520)


In [99]:
ferric_soret_total.peek()

In [ ]:
times = [-0.1,0.1, 0.5, 1., 5, 5, 50]
traces = []
plt.close('all')
fig = plt.figure()
ax = fig.gca()
colours = np.flipud(plt.cm.viridis(np.linspace(0, 1, len(times))))

insax = ax.inset_axes([0.38, 0.5, 0.4, 0.4])
for idx, time in enumerate(times):
    ax.plot(ferric_soret_total_cut.wavelengths, 
             ferric_soret_total_cut.get_wavelength_trace(time), label=str(time)+'ps', color=colours[idx])


    insax.plot(ferric_soret_total_cut.wavelengths, 
             ferric_soret_total_cut.get_wavelength_trace(time), label=str(time)+'ps', color=colours[idx])


insax.set_xlim(520, 640)
insax.set_ylim(-0.5,1.5)
insax.set_yticks([0, 1])
insax.set_xlabel('Wavelength (nm)')
insax.set_ylabel(r'$\Delta OD$')
insax.set_xticks([520, 580, 640])
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\Delta OD$')
ax.set_xlim(350,700)
ax.set_ylim(-3,12)
ax.legend()



In [ ]:
ferric_soret_total.compute_SVD()

In [ ]:
%autoreload 1
plt.close('all')
plt.figure()
sing = ferric_soret_total.singular_values
for idx, spectrum in enumerate(ferric_soret_total.principal_spectra[0:3]):
    rp.RugTools.cut_wavelengths_spectrum(ferric_soret_total.wavelengths, spectrum, 510, 520)
    #rp.RugTools.cut_wavelengths_spectrum(ferric_soret_total.wavelengths, spectrum, 405, 415)
    plt.plot(ferric_soret_total.wavelengths, spectrum, label='Component '+str(idx+1))

plt.legend()
plt.xlabel('Wavelength (nm)')
plt.ylabel(r'$\Delta OD$')
plt.xlim(350, 700)

In [ ]:
plt.figure()
plt.plot(ferric_soret_total.times, ferric_soret_total.get_time_trace(410))
plt.plot(solvent_soret_shg.times, solvent_soret_shg.get_time_trace(410))


In [ ]:
ferric_soret_total.peek(min_max=(-10, 1))